In [13]:
import pandas as pd
import pickle
df = pd.read_csv("card_transdata.csv")
df

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
999995,2.207101,0.112651,1.626798,1.0,1.0,0.0,0.0,0.0
999996,19.872726,2.683904,2.778303,1.0,1.0,0.0,0.0,0.0
999997,2.914857,1.472687,0.218075,1.0,1.0,0.0,1.0,0.0
999998,4.258729,0.242023,0.475822,1.0,0.0,0.0,1.0,0.0


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = train_test_split(
    df[["distance_from_home", "distance_from_last_transaction", "repeat_retailer", "used_chip", "used_pin_number", "online_order"]],
    df.fraud, 
    test_size=0.2,
    random_state=42)

def dataprep(data):
    scaler = StandardScaler()
    catDFH = []
    distance_from_home = data['distance_from_home'].fillna(data['distance_from_home'].mean())
    distance_from_home.index = range(len(data.index))
    for i in range(data.shape[0]) :
        if distance_from_home[i] <= 10:
            catDFH.append("proche")
        elif distance_from_home[i] > 10 and distance_from_home[i] <= 60:
            catDFH.append("moyen")
        elif distance_from_home[i] > 60 and distance_from_home[i] <= 500:
            catDFH.append("loin")
        else:
            catDFH.append("étranger")
    #print(pd.DataFrame(catDFH, columns = ['catDFH'])['catDFH'].value_counts())
    distance_from_home = pd.get_dummies(pd.DataFrame(catDFH, columns = ['catDFH']), prefix='distance_from_home')
    
    catDFLT = []
    distance_from_last_transaction = data['distance_from_last_transaction'].fillna(data['distance_from_last_transaction'].mean())
    distance_from_last_transaction.index = range(len(data.index))
    for i in range(data.shape[0]) :
        if distance_from_last_transaction[i] <= 10:
            catDFLT.append("proche")
        elif distance_from_last_transaction[i] > 10 and distance_from_last_transaction[i] <= 60:
            catDFLT.append("moyen")
        elif distance_from_last_transaction[i] > 60 and distance_from_last_transaction[i] <= 500:
            catDFLT.append("loin")
        else:
            catDFLT.append("étranger")
    #print(pd.DataFrame(catDFLT, columns = ['catDFLT'])['catDFLT'].value_counts())
    distance_from_last_transaction = pd.get_dummies(pd.DataFrame(catDFLT, columns = ['catDFLT']), prefix='distance_from_last_transaction')
    
    dpFinal = distance_from_home.join(distance_from_last_transaction).join(data[['repeat_retailer']]).join(data[['used_chip']]).join(data[['used_pin_number']]).join(data[['online_order']])
    dpFinal = scaler.fit_transform(dpFinal)
    return dpFinal

dataprep(x_test)
dataprep(x_train)

rf = RandomForestClassifier(n_estimators=250, random_state=4, max_features=2)
rf.fit(x_train, y_train)
pickle.dump(rf, open('model.pkl', 'wb'))

In [11]:
rf.score(x_test, y_test)

0.99999

In [12]:
rf.score(x_train, y_train)

1.0

In [6]:
rf.predict(x_test)

array([0., 0., 0., ..., 1., 0., 0.])